In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [5]:
# Load the csv exported in Part I to a DataFrame
weather_df = pd.read_csv('../Resources/weather_df.csv')
weather_df

,Unnamed: 0,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Hermanus,90,ZA,82,-34.42,19.23,62.01,4.00
1,1,Dubti,0,ET,57,11.74,41.09,76.21,6.51
2,2,Amuntai,1,ID,88,-2.42,115.25,77.36,0.83
3,3,Yangjiang,84,CN,50,21.85,111.97,69.19,7.94
4,4,Atuona,85,PF,76,-9.80,-139.03,77.61,15.79
...,...,...,...,...,...,...,...,...,...
549,549,Necochea,89,AR,82,-38.55,-58.74,66.00,11.99
550,550,Plaster Rock,90,CA,60,46.91,-67.39,39.20,3.36
551,551,Sur,0,OM,51,22.57,59.53,73.27,6.80
552,552,Bukachacha,100,RU,93,52.98,116.92,23.13,1.97


Create a heat map that displays the humidity for every city from the part I of the homework.

In [33]:
# Grab locations and humidity from weather_df
coord = weather_df[["Latitude", "Longitude"]]
hum = weather_df["Humidity"]


fig = gmaps.figure(zoom_level= 3, center= (35, -20))
heat_layer = gmaps.heatmap_layer(coord, weights= hum, dissipating= False, max_intensity= 100,
                                 point_radius= 1)


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

Narrow down the DataFrame to find your ideal weather condition. For example:


A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.


Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [38]:
# Create criteria for the perfect vacation climate

best_conditions_df = pd.DataFrame(weather_df, columns = ["City", "Max Temperature", "Wind Speed", "Cloudiness"])

good_max_temp = (best_conditions_df["Max Temperature"] <= 80) & (best_conditions_df["Max Temperature"] > 70)
good_wind_spd = best_conditions_df["Wind Speed"] < 10
good_cloudiness = best_conditions_df["Cloudiness"] == 0

best_conditions_df[good_max_temp & good_wind_spd & good_cloudiness]

,City,Max Temperature,Wind Speed,Cloudiness
1,Dubti,76.21,6.51,0
46,Airai,71.02,1.25,0
49,Juan de Ayolas,76.98,6.02,0
84,Salalah,77.00,2.24,0
111,Bogale,73.60,7.49,0
144,Mattru,74.64,2.06,0
148,Araouane,77.97,6.26,0
213,Flinders,72.00,3.00,0
256,Yulara,78.80,8.05,0
305,Faanui,79.90,9.89,0


Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.